In [ ]:
import numpy as np 
import pandas as pd 
import os
import json

################################### Data Processing ###############################################

def process(data, parameters, job_id: str):

    df = pd.read_csv('export-0000 (1).csv')
    #df = pd.read_csv(data[0]['downloaded'][0], sep=',')

    df = df.loc[(df.formulaName.str.contains('Formula')) & (df.resourceid.isnull() == False), :]

    metadata = [
        'resourceid', 'formulaName', 'fullFormulaNumber', 
        'maturityState', 'owner', 'creationDate', 'modificationDate', 
        'Trial code', 'Process', 'Collection'
    ]

    ids = ['formulaName', 'Trial code']


    composition = [
        '%Water',
        '%Condensat',
        '%PLE',
        '%MPC 80',
        '%Milk Powder',
        '% Rennet casein',
        '%Acid casein',
        '%Lactic Acid',
        '%Anhydrci Citric Acid',
        '%PS 35',
        '%Maasdam',
        '%Baby Cheese',
        '%Cheddar Cheese',
        '(%) Gouda Cheese',
        '%Cheese Cat 2',
        '%Cheese Cat 1',
        '%Swiss Cheese',
        '%Downgraded Pressed Paste Cat 1',
        '%Downgraded Pressed Paste Cat 2',
        '(%) PP25',
        '(%)  PP50',
        '%Butter',
        '%Melting Salt P50',
        '%Melting Salt K 2285',
        '%Fine Salt Refined Dry',
        '%DIHYDRATE TRISODIUM CITRATE',
        '%CALCIUM CONCENTRATE',
        '%Starch',
        '%RMC 80 HIGH HEAT',
        '%TEXTURING AGENT FLANOGEN',
        '%SICAMELT NSQ23',
        '%SICAMELT C23',
        '%C23 NF',
        '%CCL',
        '%CLL.th',
        'Vitamine D'
    ]

    thermomechanical = [
        'D+20',
        'D+7',
        'L',
        'a',
        'b'
    ]

    sensory = [
        'Taste Rate',
        'Texture Rate',
        'Firmness Rate',
        'Appearance Rate',
        'Irritancy Rate'
    ]

    # Fill NaN values
    df[composition] = df[composition].replace([np.NaN, '', ' ', '  '], 0).astype(float)

    # # Concatenate composition and thermomechanical
    df_thermomechanical = pd.concat((df[metadata], df[composition], df[thermomechanical]), axis=1)

    from sklearn.ensemble import GradientBoostingRegressor

    def fillna(df, X_cols, to_fill):
        for col in to_fill:
            gb = GradientBoostingRegressor().fit(df.dropna()[X_cols], df.dropna()[col])
            df.loc[df[col].isna(), col] = gb.predict(df.loc[df[col].isna(), X_cols])
        return df


    df_thermomechanical = fillna(df_thermomechanical, X_cols=composition, to_fill=thermomechanical)
    df_thermomechanical.drop_duplicates(subset=ids, inplace=True)


    df_sensory = pd.concat((df[metadata], df[composition], df[sensory].fillna(0)), axis=1)


    path_thermo = os.path.abspath('data_trials_thermomechanical.csv')
    path_sensory = os.path.abspath('data_trials_sensory.csv')

    df_thermomechanical.to_csv(path_thermo, index=False)
    df_sensory.to_csv(path_sensory, index=False)



    return [
        [
            {
                "file_path": path_thermo,
                "file_id": "formulasRDF/processed/lot1/thermomechanical/data_transformed_thermomechanical.csv",
                "bucket_name": "belstoragebucket"
            }
        ],

        [
            {
                "file_path": df_sensory,
                "file_id": "formulasRDF/processed/lot1/sensory/data_transformed_sensory.csv",
                "bucket_name": "belstoragebucket"
            }
        ]
        ]

In [3]:
import numpy as np 
import pandas as pd 
import os
import json

################################### Data Processing ###############################################

In [4]:
df = pd.read_csv('export-0000 (1).csv')
#df = pd.read_csv(data[0]['downloaded'][0], sep=',')

df = df.loc[(df.formulaName.str.contains('Formula')) & (df.resourceid.isnull() == False), :]

metadata = [
    'resourceid', 'formulaName', 'fullFormulaNumber', 
    'maturityState', 'owner', 'creationDate', 'modificationDate', 
    'Trial code', 'Process', 'Collection'
]

ids = ['formulaName', 'Trial code']


composition = [
    '%Water',
    '%Condensat',
    '%PLE',
    '%MPC 80',
    '%Milk Powder',
    '% Rennet casein',
    '%Acid casein',
    '%Lactic Acid',
    '%Anhydrci Citric Acid',
    '%PS 35',
    '%Maasdam',
    '%Baby Cheese',
    '%Cheddar Cheese',
    '(%) Gouda Cheese',
    '%Cheese Cat 2',
    '%Cheese Cat 1',
    '%Swiss Cheese',
    '%Downgraded Pressed Paste Cat 1',
    '%Downgraded Pressed Paste Cat 2',
    '(%) PP25',
    '(%)  PP50',
    '%Butter',
    '%Melting Salt P50',
    '%Melting Salt K 2285',
    '%Fine Salt Refined Dry',
    '%DIHYDRATE TRISODIUM CITRATE',
    '%CALCIUM CONCENTRATE',
    '%Starch',
    '%RMC 80 HIGH HEAT',
    '%TEXTURING AGENT FLANOGEN',
    '%SICAMELT NSQ23',
    '%SICAMELT C23',
    '%C23 NF',
    '%CCL',
    '%CLL.th',
    'Vitamine D'
]

thermomechanical = [
    'D+20',
    'D+7',
    'L',
    'a',
    'b'
]

sensory = [
    'Taste Rate',
    'Texture Rate',
    'Firmness Rate',
    'Appearance Rate',
    'Irritancy Rate'
]

# Fill NaN values
df[composition] = df[composition].replace([np.NaN, '', ' ', '  '], 0).astype(float)

# # Concatenate composition and thermomechanical
df_thermomechanical = pd.concat((df[metadata], df[composition], df[thermomechanical]), axis=1)

from sklearn.ensemble import GradientBoostingRegressor

def fillna(df, X_cols, to_fill):
    for col in to_fill:
        gb = GradientBoostingRegressor().fit(df.dropna()[X_cols], df.dropna()[col])
        df.loc[df[col].isna(), col] = gb.predict(df.loc[df[col].isna(), X_cols])
    return df


df_thermomechanical = fillna(df_thermomechanical, X_cols=composition, to_fill=thermomechanical)
df_thermomechanical.drop_duplicates(subset=ids, inplace=True)


path = os.path.abspath('data_trials_thermomechanical.csv')
df_thermomechanical.to_csv(path, index=False)
    

In [5]:
df_thermomechanical

,resourceid,formulaName,fullFormulaNumber,maturityState,owner,creationDate,modificationDate,Trial code,Process,Collection,...,%SICAMELT C23,%C23 NF,%CCL,%CLL.th,Vitamine D,D+20,D+7,L,a,b
87,uuid:9651b1e6-eaa1-496a-ad38-769b7e4cb448,Formula 26,FORM-00000407-I1,Draft,Laure BUQUEN,2024-08-23T20:16:50,2024-08-26T08:45:36,VQR Justin E13,Post Acidification-precooking 83C,1.0,...,2.25,0.0,1.06,1.0,0.0,13.600000,27.150000,87.720000,-2.500000,15.950000
88,uuid:dcd69759-48c9-4b38-97f9-7eff396aa620,Formula 25,FORM-00000406-I1,Draft,Laure BUQUEN,2024-08-23T20:16:45,2024-08-27T04:39:09,VQR Justin E12.2,Post Acidification-precooking 83C,2.0,...,1.75,0.0,1.05,1.0,0.0,7.650000,20.000000,88.480000,-2.760000,15.740000
89,uuid:b4a12444-fffb-46f1-97ac-412bcc5fc0a9,Formula 24,FORM-00000405-I1,Draft,Laure BUQUEN,2024-08-23T20:16:41,2024-08-26T08:44:00,VQR Justin E12.1,Post Acidification-precooking 83C,1.0,...,2.25,0.0,1.06,1.0,0.0,9.750000,26.100000,88.000000,-2.700000,15.600000
90,uuid:f5b64d9b-fa17-48db-b71c-e8ac5958404e,Formula 23,FORM-00000404-I1,Draft,Laure BUQUEN,2024-08-23T20:16:36,2024-08-26T08:43:32,VQR Justin E11.2,Acidification at the end of premix phase-preco...,2.0,...,1.75,0.0,1.06,1.3,0.0,14.400000,44.050000,92.210000,-2.230000,17.440000
92,uuid:4eb29942-9c38-453a-ac4b-363c5fd93a2e,Formula 22,FORM-00000403-I1,Draft,Laure BUQUEN,2024-08-23T20:16:32,2024-08-26T08:42:29,VQR Justin E11.1,Post Acidification-precooking 83C,2.0,...,2.25,0.0,1.06,1.3,0.0,15.300000,40.033333,91.940000,-2.220000,17.470000
94,uuid:f9c70f39-587a-446d-bb4f-5db016416984,Formula 21,FORM-00000402-I1,Draft,Laure BUQUEN,2024-08-23T20:16:27,2024-08-26T08:41:49,VQR Justin E10.2,Acidification at the end of premix phase-preco...,2.0,...,2.25,0.0,1.10,1.5,0.0,12.567425,30.145438,91.093738,-2.708291,16.681187
95,uuid:d801864a-4a18-44b5-863b-0a2cadad28d5,Formula 20,FORM-00000401-I1,Draft,Laure BUQUEN,2024-08-23T20:16:23,2024-08-26T08:41:05,VQR Justin E10.1,Post Acidification-precooking 83C,1.0,...,2.25,0.0,1.10,1.5,0.0,12.567425,30.145438,91.093738,-2.708291,16.681187
96,uuid:b8c1ec8e-b2fb-4500-84a1-a5239fae1876,Formula 19,FORM-00000400-I1,Draft,Laure BUQUEN,2024-08-23T20:16:18,2024-08-26T08:40:22,VQR Justin E09.2,Post Acidification-precooking 83C,2.0,...,2.25,0.0,1.05,1.5,0.0,14.400000,31.950000,88.490000,-2.650000,16.340000
97,uuid:d0300235-9734-418f-9f2c-25c07ac3cfaf,Formula 18,FORM-00000399-I1,Draft,Laure BUQUEN,2024-08-23T20:16:14,2024-08-26T08:39:00,VQR Justin E09.1,Post Acidification-precooking 85C,1.0,...,2.25,0.0,1.05,1.5,0.0,13.000000,25.050000,92.450000,-2.520000,17.550000
98,uuid:0351365a-134b-4bee-a8bd-97928ac75bd4,Formula 17,FORM-00000398-I1,Draft,Laure BUQUEN,2024-08-23T20:16:09,2024-08-26T08:35:25,VQR Justin E08.2,Post Acidification,2.0,...,2.25,0.0,0.99,1.5,0.0,14.444028,45.406249,90.931494,-2.243904,18.186136


In [6]:
df_thermomechanical.columns[-1]dd

'b'